# ⚽ Modélisation et Analyse de Données Footballistiques

* Les imports : 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



***

## 1️⃣ Web Scraping

### ➕ Mise en place de l’environnement de scraping

In [2]:

def initialiser_driver():
    
    try:
        driver = webdriver.Chrome()
        print("Driver initialisé avec succès en mode headless.")
        return driver
    except Exception as e:
        print(f"Erreur lors de l'initialisation du driver : {e}")
        return None

In [3]:
# driver = initialiser_driver()
driver = webdriver.Chrome()
driver.get("https://fbref.com/en/comps/9/history/Premier-League-Seasons")
print(driver.title)

Premier League Seasons | FBref.com


### ➕ Collecte des informations sur les équipes

* clicker sur le season 2024-2025

In [4]:

wait = WebDriverWait(driver, 10)
last_season = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, '2024-2025')))
# driver.execute_script("arguments[0].scrollIntoView(true);", last_season)
# last_season.click()
url = last_season.get_attribute("href")
driver.get(url)



* trouver le tableau de teams 

In [5]:
teams_table = driver.find_element(By.CSS_SELECTOR, '#stats_squads_standard_for')
# print(teams_table.text)


In [6]:


teams_table = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "table#results2024-202591_overall"))
)

# attendre que la première cellule de la colonne "team" soit visible
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "table#results2024-202591_overall tbody tr td.left a"))
)


<selenium.webdriver.remote.webelement.WebElement (session="1eeb162d47ddd30b546bd9473b4e0dcd", element="f.2959276231AD02ACDA60A80EA767DEDC.d.285E9496D48AEFFD7E3FD530EFA69582.e.68")>

* afficher les nom des équipes : 

In [7]:

# extraire tous les liens et noms
teams_links_names = []
teams_elements = teams_table.find_elements(By.CSS_SELECTOR, "tbody tr td.left a")
# teams_names = []
for el in teams_elements:
    name = el.text.strip()
    href = el.get_attribute("href")
    teams_links_names.append((name, href))
    # teams_names.append(name)
    print("✅ Équipe trouvée :", name)

print(f"\nTotal équipes trouvées : {len(teams_links_names)}\n")


✅ Équipe trouvée : Liverpool
✅ Équipe trouvée : Arsenal
✅ Équipe trouvée : Manchester City
✅ Équipe trouvée : Chelsea
✅ Équipe trouvée : Newcastle Utd
✅ Équipe trouvée : Aston Villa
✅ Équipe trouvée : Nott'ham Forest
✅ Équipe trouvée : Brighton
✅ Équipe trouvée : Bournemouth
✅ Équipe trouvée : Brentford
✅ Équipe trouvée : Fulham
✅ Équipe trouvée : Crystal Palace
✅ Équipe trouvée : Everton
✅ Équipe trouvée : West Ham
✅ Équipe trouvée : Manchester Utd
✅ Équipe trouvée : Wolves
✅ Équipe trouvée : Tottenham
✅ Équipe trouvée : Leicester City
✅ Équipe trouvée : Ipswich Town
✅ Équipe trouvée : Southampton

Total équipes trouvées : 20



#### 📍 Extraire les informations sur les équipes participant à la Premier League pour la saison 2024–2025.
* Les matchs joués durant la saison, avec leurs statistiques clés.

In [8]:

info_team_thead = []
info_teams = []

for name, link in teams_links_names:
    driver.get(link)

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table#matchlogs_for"))
    )

    headers = driver.find_elements(By.CSS_SELECTOR, "table#matchlogs_for thead tr th")

    current_headers = []
    current_headers.append('Squad')
    for th in headers:
        data_stat = th.get_attribute("data-stat")

        if data_stat == "referee":
            current_headers.append(th.text.strip())
            break

        if data_stat not in ["match_report", "notes"]:
            current_headers.append(th.text.strip())

    info_team_thead = current_headers

    rows = driver.find_elements(By.CSS_SELECTOR, "table#matchlogs_for tbody tr")

    team_data = []
    for row in rows:
        if "thead" in row.get_attribute("class"):
            continue

        cell_date = row.find_element(By.CSS_SELECTOR, "th")
        cells = row.find_elements(By.CSS_SELECTOR, "td")

        if len(cells) == 0:
            continue

        row_values = []
        row_values.append(name)
        
        date_data_stats = cell_date.get_attribute("data-stat")
        if date_data_stats == 'date':
            row_values.append(cell_date.text.strip())
        
        for td in cells:
            data_stat = td.get_attribute("data-stat")

            if data_stat == "referee":
                row_values.append(td.text.strip())
                break

            if data_stat not in ["match_report", "notes"]:
                row_values.append(td.text.strip())

        if row_values:
            team_data.append(row_values)

    info_teams.append({
        "url": link,
        "headers": info_team_thead,
        "data": team_data
    })

    print(f"\n✅ {len(team_data)} lignes valides extraites depuis {link}")

    driver.back()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table#results2024-202591_overall"))
    )



✅ 56 lignes valides extraites depuis https://fbref.com/en/squads/822bd0ba/2024-2025/Liverpool-Stats

✅ 58 lignes valides extraites depuis https://fbref.com/en/squads/18bb7c10/2024-2025/Arsenal-Stats

✅ 57 lignes valides extraites depuis https://fbref.com/en/squads/b8fd03ef/2024-2025/Manchester-City-Stats

✅ 57 lignes valides extraites depuis https://fbref.com/en/squads/cff3d9bb/2024-2025/Chelsea-Stats

✅ 48 lignes valides extraites depuis https://fbref.com/en/squads/b2b47a98/2024-2025/Newcastle-United-Stats

✅ 57 lignes valides extraites depuis https://fbref.com/en/squads/8602292d/2024-2025/Aston-Villa-Stats

✅ 44 lignes valides extraites depuis https://fbref.com/en/squads/e4a775cb/2024-2025/Nottingham-Forest-Stats

✅ 45 lignes valides extraites depuis https://fbref.com/en/squads/d07537b9/2024-2025/Brighton-and-Hove-Albion-Stats

✅ 43 lignes valides extraites depuis https://fbref.com/en/squads/4ba7cbea/2024-2025/Bournemouth-Stats

✅ 43 lignes valides extraites depuis https://fbref.com

In [9]:
# afficher les headers des table de matches (informations sur les teams)
print(f"Les headers de tables des matches : \n{info_team_thead}")

Les headers de tables des matches : 
['Squad', 'Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation', 'Opp Formation', 'Referee']


In [10]:
# afficher les links + headeer + data de chaque teams
print(f"LEs infos sur teams : \n{info_teams}")

LEs infos sur teams : 
[{'url': 'https://fbref.com/en/squads/822bd0ba/2024-2025/Liverpool-Stats', 'headers': ['Squad', 'Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation', 'Opp Formation', 'Referee'], 'data': [['Liverpool', '2024-08-17', '12:30', 'Premier League', 'Matchweek 1', 'Sat', 'Away', 'W', '2', '0', 'Ipswich Town', '2.6', '0.5', '62', '30,014', 'Virgil van Dijk', '4-2-3-1', '4-2-3-1', 'Tim Robinson'], ['Liverpool', '2024-08-25', '16:30', 'Premier League', 'Matchweek 2', 'Sun', 'Home', 'W', '2', '0', 'Brentford', '2.5', '0.5', '62', '60,017', 'Virgil van Dijk', '4-2-3-1', '4-4-2', 'Stuart Attwell'], ['Liverpool', '2024-09-01', '16:00', 'Premier League', 'Matchweek 3', 'Sun', 'Away', 'W', '3', '0', 'Manchester Utd', '1.8', '1.4', '47', '73,738', 'Virgil van Dijk', '4-2-3-1', '4-2-3-1', 'Anthony Taylor'], ['Liverpool', '2024-09-14', '15:00', 'Premier League', 'Matchweek 4', 'Sat', 'Home', 'L'

#### 📍 Extraire les competitions d'après la table des matches : 

In [11]:
list_com = []
print(f"Le nombre total des teams : {len(info_teams)}")
for i in range(len(info_teams)):
    n = len(info_teams[i]['data'])
    # print(n)
    for j in range(n):
        list_com.append(info_teams[i]['data'][j][3])
        
print("\navant supprimer les doubeleons dans les compétition : le nombre total des competitions est ====>", len(list_com))
sets_com = set(list_com)
print(list_com)
competitions =list(sets_com)
print("\napres supprimer les doubeleons dans les compétition : le nombre total des competitions est ====>", len(competitions))
print(competitions)

Le nombre total des teams : 20

avant supprimer les doubeleons dans les compétition : le nombre total des competitions est ====> 975
['Premier League', 'Premier League', 'Premier League', 'Premier League', 'Champions Lg', 'Premier League', 'EFL Cup', 'Premier League', 'Champions Lg', 'Premier League', 'Premier League', 'Champions Lg', 'Premier League', 'EFL Cup', 'Premier League', 'Champions Lg', 'Premier League', 'Premier League', 'Champions Lg', 'Premier League', 'Premier League', 'Champions Lg', 'Premier League', 'EFL Cup', 'Premier League', 'Premier League', 'Premier League', 'Premier League', 'EFL Cup', 'FA Cup', 'Premier League', 'Premier League', 'Champions Lg', 'Premier League', 'Champions Lg', 'Premier League', 'EFL Cup', 'FA Cup', 'Premier League', 'Premier League', 'Premier League', 'Premier League', 'Premier League', 'Champions Lg', 'Premier League', 'Champions Lg', 'EFL Cup', 'Premier League', 'Premier League', 'Premier League', 'Premier League', 'Premier League', 'Premier

In [12]:
# restrecturer les datas comme un list des liste des row de tables des matches

datas_teams = []
print(f"Nombre total des equipes : {len(info_teams)}")
for i in range(len(info_teams)):
    n = len(info_teams[i]['data'])
    # print(n)
    for j in range(n):
        datas_teams.append(info_teams[i]['data'][j])
       
print(datas_teams)
# print(datas_teams[0])
# datas_teams

Nombre total des equipes : 20
[['Liverpool', '2024-08-17', '12:30', 'Premier League', 'Matchweek 1', 'Sat', 'Away', 'W', '2', '0', 'Ipswich Town', '2.6', '0.5', '62', '30,014', 'Virgil van Dijk', '4-2-3-1', '4-2-3-1', 'Tim Robinson'], ['Liverpool', '2024-08-25', '16:30', 'Premier League', 'Matchweek 2', 'Sun', 'Home', 'W', '2', '0', 'Brentford', '2.5', '0.5', '62', '60,017', 'Virgil van Dijk', '4-2-3-1', '4-4-2', 'Stuart Attwell'], ['Liverpool', '2024-09-01', '16:00', 'Premier League', 'Matchweek 3', 'Sun', 'Away', 'W', '3', '0', 'Manchester Utd', '1.8', '1.4', '47', '73,738', 'Virgil van Dijk', '4-2-3-1', '4-2-3-1', 'Anthony Taylor'], ['Liverpool', '2024-09-14', '15:00', 'Premier League', 'Matchweek 4', 'Sat', 'Home', 'L', '0', '1', "Nott'ham Forest", '0.9', '0.4', '68', '60,344', 'Virgil van Dijk', '4-2-3-1', '4-2-3-1', 'Michael Oliver'], ['Liverpool', '2024-09-17', '21:00 (20:00)', 'Champions Lg', 'League phase', 'Tue', 'Away', 'W', '3', '1', 'it Milan', '3.1', '0.6', '51', '59,826'

### 📍 Enregistrer les données de équipes (les matches) dans un fichier CSV

In [ ]:

df_matches_ = pd.DataFrame(datas_teams, columns=info_team_thead)

# Sauvegarde au format CSV
df_matches_.to_csv('../data/data_scraper/matchs_infos.csv', index=False, encoding='utf-8')

print("✅ Fichier CSV enregistré avec succès !")


✅ Fichier CSV enregistré avec succès !


* 📍 Lire les informations des matches depuis le fichier CSV sauvegarder (dataframe)

In [ ]:
df_matches = pd.read_csv('../data/data_scraper/matchs_infos.csv')
print(df_matches.duplicated().sum())

0


In [ ]:
df_matches.head(5)

,Squad,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee
0,Liverpool,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2,0,Ipswich Town,2.6,0.5,62.0,"30,014",Virgil van Dijk,4-2-3-1,4-2-3-1,Tim Robinson
1,Liverpool,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2,0,Brentford,2.5,0.5,62.0,"60,017",Virgil van Dijk,4-2-3-1,4-4-2,Stuart Attwell
2,Liverpool,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3,0,Manchester Utd,1.8,1.4,47.0,"73,738",Virgil van Dijk,4-2-3-1,4-2-3-1,Anthony Taylor
3,Liverpool,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0,1,Nott'ham Forest,0.9,0.4,68.0,"60,344",Virgil van Dijk,4-2-3-1,4-2-3-1,Michael Oliver
4,Liverpool,2024-09-17,21:00 (20:00),Champions Lg,League phase,Tue,Away,W,3,1,it Milan,3.1,0.6,51.0,"59,826",Virgil van Dijk,4-2-3-1,4-2-3-1,Espen Eskås


### ➕ Collecte des informations sur les joueurs

In [16]:

headers_players = []
data_players = []

for name, link in teams_links_names:
    driver.get(link)

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table#stats_standard_9"))
    )

    headers = driver.find_elements(By.CSS_SELECTOR, "table#stats_standard_9 thead tr:not(.over_header) th")

    current_headers = []
    current_headers.append('Squad')
    for th in headers:
        data_stat = th.get_attribute("data-stat")

        if data_stat == "cards_red":
            current_headers.append(th.text.strip())
            break

        current_headers.append(th.text.strip())

    headers_players = current_headers

    rows = driver.find_elements(By.CSS_SELECTOR, "table#stats_standard_9 tbody tr")

    team_player_data = []
    for row in rows:

        if "over_header thead" in row.get_attribute("class"):
            continue
        if "thead" in row.get_attribute("class"):
            continue


        cell_date = row.find_element(By.CSS_SELECTOR, "th")
        cells = row.find_elements(By.CSS_SELECTOR, "td")

        if len(cells) == 0:
            continue

        row_values = []
        row_values.append(name)
        
        player_data_stats = cell_date.get_attribute("data-stat")
        if player_data_stats == 'player':
            row_values.append(cell_date.text.strip())
        
        for td in cells:
            data_stat = td.get_attribute("data-stat")

            if data_stat == "cards_red":
                row_values.append(td.text.strip())
                break

            row_values.append(td.text.strip())

        if row_values:
            team_player_data.append(row_values)

    data_players.append({
        "url": link,
        "headers": headers_players,
        "data": team_player_data
    })

    print(f"\n✅ {len(team_player_data)} lignes valides extraites depuis {link}")

    driver.back()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table#results2024-202591_overall"))
    )



✅ 29 lignes valides extraites depuis https://fbref.com/en/squads/822bd0ba/2024-2025/Liverpool-Stats

✅ 38 lignes valides extraites depuis https://fbref.com/en/squads/18bb7c10/2024-2025/Arsenal-Stats

✅ 36 lignes valides extraites depuis https://fbref.com/en/squads/b8fd03ef/2024-2025/Manchester-City-Stats

✅ 35 lignes valides extraites depuis https://fbref.com/en/squads/cff3d9bb/2024-2025/Chelsea-Stats

✅ 30 lignes valides extraites depuis https://fbref.com/en/squads/b2b47a98/2024-2025/Newcastle-United-Stats

✅ 34 lignes valides extraites depuis https://fbref.com/en/squads/8602292d/2024-2025/Aston-Villa-Stats

✅ 27 lignes valides extraites depuis https://fbref.com/en/squads/e4a775cb/2024-2025/Nottingham-Forest-Stats

✅ 42 lignes valides extraites depuis https://fbref.com/en/squads/d07537b9/2024-2025/Brighton-and-Hove-Albion-Stats

✅ 38 lignes valides extraites depuis https://fbref.com/en/squads/4ba7cbea/2024-2025/Bournemouth-Stats

✅ 35 lignes valides extraites depuis https://fbref.com

In [34]:
print(f"les headers des players : {data_players[0]['headers']}")

les headers des players : ['Squad', 'Player', 'Nation', 'Pos', 'Age', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR']


In [36]:
print(f"Le nombres des headers des players = {len(data_players[3]['headers'])}")
print(f"Le nombre des colonnes dans data des players = {len(data_players[3]['data'][20])}")

Le nombres des headers des players = 17
Le nombre des colonnes dans data des players = 17


In [37]:
# restrecturer les datas comme un list des liste des row de tables des players

datas_players = []
print(f"Nombre total des equipes : {len(data_players)}")
for i in range(len(data_players)):
    n = len(data_players[i]['data'])
    # print(n)
    for j in range(n):
        datas_players.append(data_players[i]['data'][j])
       
print(datas_players)
# print(datas_players[0])
# datas_players

Nombre total des equipes : 20
[['Liverpool', 'Mohamed Salah', 'eg EGY', 'FW', '32', '38', '38', '3,371', '37.5', '29', '18', '47', '20', '9', '9', '1', '0'], ['Liverpool', 'Virgil van Dijk', 'nl NED', 'DF', '33', '37', '37', '3,330', '37.0', '3', '1', '4', '3', '0', '0', '5', '0'], ['Liverpool', 'Ryan Gravenberch', 'nl NED', 'MF', '22', '37', '37', '3,160', '35.1', '0', '4', '4', '0', '0', '0', '6', '1'], ['Liverpool', 'Alexis Mac Allister', 'ar ARG', 'MF', '25', '35', '30', '2,599', '28.9', '5', '5', '10', '5', '0', '0', '6', '0'], ['Liverpool', 'Ibrahima Konaté', 'fr FRA', 'DF', '25', '31', '30', '2,560', '28.4', '1', '2', '3', '1', '0', '0', '5', '0'], ['Liverpool', 'Dominik Szoboszlai', 'hu HUN', 'MF', '23', '36', '29', '2,491', '27.7', '6', '6', '12', '6', '0', '0', '6', '0'], ['Liverpool', 'Andrew Robertson', 'sct SCO', 'DF', '30', '33', '29', '2,482', '27.6', '0', '1', '1', '0', '0', '0', '3', '1'], ['Liverpool', 'Alisson', 'br BRA', 'GK', '31', '28', '28', '2,508', '27.9', '0',

### 📍 Enregistrer les données de players dans un fichier CSV

In [ ]:

df_players_ = pd.DataFrame(datas_players, columns=headers_players)

# Sauvegarde au format CSV
df_players_.to_csv('../data/data_scraper/players_infos.csv', index=False, encoding='utf-8')

print("✅ Fichier CSV enregistré avec succès !")


✅ Fichier CSV enregistré avec succès !


* 📍 Lire les informations des players depuis le fichier CSV sauvegarder (dataframe)

In [ ]:
df_player = pd.read_csv('../data/data_scraper/players_infos.csv')
print(df_players_.dupicated().sum())

0


In [41]:
df_player.head(5)

,Squad,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR
0,Liverpool,Mohamed Salah,eg EGY,FW,32.0,38,38,"3,371",37.5,29.0,18.0,47.0,20.0,9.0,9.0,1.0,0.0
1,Liverpool,Virgil van Dijk,nl NED,DF,33.0,37,37,"3,330",37.0,3.0,1.0,4.0,3.0,0.0,0.0,5.0,0.0
2,Liverpool,Ryan Gravenberch,nl NED,MF,22.0,37,37,"3,160",35.1,0.0,4.0,4.0,0.0,0.0,0.0,6.0,1.0
3,Liverpool,Alexis Mac Allister,ar ARG,MF,25.0,35,30,"2,599",28.9,5.0,5.0,10.0,5.0,0.0,0.0,6.0,0.0
4,Liverpool,Ibrahima Konaté,fr FRA,DF,25.0,31,30,"2,560",28.4,1.0,2.0,3.0,1.0,0.0,0.0,5.0,0.0
